In [1]:
import modin.pandas as pd
import numpy as np
import src.model_builder as mb
from tqdm.notebook import tqdm

In [2]:
import ray
ray.init()

2021-08-25 10:34:00,293	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '129.240.3.22',
 'raylet_ip_address': '129.240.3.22',
 'redis_address': '129.240.3.22:6379',
 'object_store_address': '/tmp/ray/session_2021-08-25_10-33-57_681585_97483/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-08-25_10-33-57_681585_97483/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-08-25_10-33-57_681585_97483',
 'metrics_export_port': 58561,
 'node_id': '2df8c5f47f6f295ed80aeed51f0cea668cdca85e5065061e18770105'}

In [29]:
ray.shutdown()

In [3]:
import pickle

In [4]:
with open("/data/xiaoral/data/cell_trt_rna_transposed.pickle", "rb") as handle:
    data_dict = pickle.load(handle)

In [9]:
genes = list(data_dict.keys())

In [7]:
with open("/data/xiaoral/data/cell_trt_meta.pickle", "rb") as handle:
    metadata_dict = pickle.load(handle)

In [8]:
metadata_dict.keys()

dict_keys(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'TCR_clonotype_id', 'TCR_chain', 'TCR_v_gene', 'TCR_j_gene', 'TCR_c_gene', 'TCR_caten', 'cdr3s_aa', 'BCR_clonotype_id', 'BCR_chain', 'BCR_v_gene', 'BCR_j_gene', 'BCR_c_gene', 'BCR_caten', 'Condition', 'Time_point', 'Trial', 'patient_number', 'RNA_snn_res.0.8', 'seurat_clusters', 'RNA_snn_res.1', 'RNA_snn_res.1.2', 'main.ids'])

In [5]:
values = np.empty((33562, 94231))

In [10]:
for i in tqdm(range(values.shape[0])):
    values[i] = data_dict[genes[i]]

  0%|          | 0/33562 [00:00<?, ?it/s]

In [11]:
val = values.T

In [12]:
df = pd.DataFrame(val, columns=genes)

Distributing <class 'numpy.ndarray'> object. This may take some time.


In [13]:
df.head()

,AL627309.1,AL627309.3,AL627309.5,AP006222.2,AL732372.1,LINC01409,FAM87B,LINC01128,LINC00115,FAM41C,...,TMEM132D,AC012414.4,AC090164.2,AC133485.2,AC005548.1,AC008543.4,AL049794.2,DEFB123,BX322557.1,RBMXL3
0,-0.037671,0.0,-0.018433,-0.039177,0.0,-0.062080,-0.011830,-0.097566,-0.005060,-0.043819,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.022179,0.0,-0.037042,-0.025707,0.0,-0.179547,-0.048030,2.065165,-0.176500,-0.056235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.027216,0.0,-0.039989,-0.014498,0.0,-0.106396,-0.016048,-0.164212,-0.085616,-0.040237,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.023800,0.0,-0.044782,-0.009735,0.0,-0.119222,-0.017781,-0.184942,-0.108913,-0.039240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.013042,0.0,-0.048707,-0.007996,0.0,-0.097011,-0.000406,-0.144852,-0.105478,-0.022218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
for key in metadata_dict:
    df[key] = metadata_dict[key]

In [15]:
df["main.ids"].replace("Cancer cells ?", "Cancer cells", inplace=True)

In [25]:
df["main.ids"].value_counts()

Fibroblasts                  24851
Cancer cells                 24409
T- & NK-cells                18199
Endothelial cells             9708
Monocytes/Macrophages         9244
B-cells                       4318
Dendritic cells               1680
B-cells / Cancer cells        1657
B-cells / Dendritic cells      165
Name: main.ids, dtype: int64

In [19]:
df["Time_point"].value_counts()

value_counts defaulting to pandas implementation.
To request implementation, send an email to feature_requests@modin.org.


2mnd        40470
4mnd        27270
Baseline    26491
Name: Time_point, dtype: int64

In [20]:
df_2mnd = df[df["Time_point"] == "2mnd"]

In [21]:
df_4mnd = df[df["Time_point"] == "4mnd"]
df_base = df[df["Time_point"] == "Baseline"]

In [23]:
df_base_cancer =df_base[df_base["main.ids"] == "Cancer cells"]

In [26]:
df_4mnd_cancer = df_4mnd[df_4mnd["main.ids"] == "Cancer cells"]
df_2mnd_cancer = df_2mnd[df_2mnd["main.ids"] == "Cancer cells"]

In [ ]:
df_2mnd_cancer.patient_number.value_counts()

value_counts defaulting to pandas implementation.
2021-08-25 11:16:37,863	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265


In [27]:
df_2mnd_cancer.to_pickle("/data/severs/new_data/Cancer_2mnd.pkl")

In [28]:
df_4mnd_cancer.to_pickle("/data/severs/new_data/Cancer_4mnd.pkl")

In [25]:
df_base_cancer.to_pickle("/data/severs/new_data/Cancer_baseline.pkl")

`DataFrame.to_pickle` defaulting to pandas implementation.
Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.


In [26]:
cancer_df = df.loc[df['main.ids'] == 'Cancer cells']

In [27]:
cancer_df.patient_num

,0,1,2,3,4,5,6,7,8,9,...,BCR_caten,Condition,Time_point,Trial,patient_number,RNA_snn_res.0.8,seurat_clusters,RNA_snn_res.1,RNA_snn_res.1.2,main.ids
NL67A_AAACCTGCACTGTTAG-1,-0.003800,0.0,-0.028163,-0.048774,0.0,-0.185302,-0.045739,-0.329466,-0.212353,-0.043305,...,NA,Tumor,Baseline,NeoLetExe,67,6,4,5,4,Cancer cells
NL67A_AAACCTGCATAAAGGT-1,-0.009624,0.0,-0.048950,-0.008252,0.0,-0.077640,0.010190,-0.110326,-0.089886,-0.013749,...,NA,Tumor,Baseline,NeoLetExe,67,11,10,12,10,Cancer cells
NL67A_AAACGGGAGATCTGAA-1,-0.023499,0.0,-0.037598,-0.018245,0.0,-0.075968,-0.001593,-0.111781,-0.056672,-0.029224,...,NA,Tumor,Baseline,NeoLetExe,67,0,0,0,0,Cancer cells
NL67A_AAACGGGAGGGCACTA-1,-0.021205,0.0,-0.038602,-0.018073,0.0,-0.079083,-0.001356,-0.117545,-0.064944,-0.027675,...,NA,Tumor,Baseline,NeoLetExe,67,0,0,0,0,Cancer cells
NL67A_AAACGGGGTGTTTGGT-1,-0.006725,0.0,-0.039103,-0.034254,0.0,-0.245807,-0.069205,-0.430010,-0.281275,-0.059379,...,NA,Tumor,Baseline,NeoLetExe,67,19,20,21,20,Cancer cells
